In [ ]:
import random
import statistics
import pandas as pd
import math
import numpy

Key scheduling algorithm 

In [ ]:
def KSA(key,rangea):
    #rangea = 1024
    keylength = len(key)
    S = [i for i in range(rangea)]
    j = 0
    for i in range(rangea):
        j = (j + S[i] + key[i % keylength]) % rangea
        S[i], S[j] = S[j], S[i]

    return S

Pseudo Random generation algorithm

In [ ]:
def PRGA(S,rangea):
    i = 0
    j = 0
    #rangea = 1024
    temp=[]
    while i<rangea-1:
        i = (i + 1) % rangea
        j = (j + S[i]) % rangea
        S[i], S[j] = S[j], S[i]  # swap

        K = S[(S[i] + S[j]) % rangea]
  
        temp.append(K)
    i=rangea-1
    i = (i + 1) % rangea
    j = (j + S[i]) % rangea
    S[i], S[j] = S[j], S[i]  # swap

    K = S[(S[i] + S[j]) % rangea]


    temp.append(K)


    return temp 

Main RC function

In [ ]:
def RC4(key,a):
    S = KSA(key,a)
    return PRGA(S,a)

Convert key to ASCII array

In [ ]:
def convert_key(s):
        return [ord(c) for c in s]

RC4 Encryption

In [ ]:
def RC4Encryption(plaintext,keystream):
  temp=[] 
  for i in range(len(plaintext)):
      #sys.stdout.write("%02X" % (ord(plaintext[i]) ^ (keystream[i]))
      aux=ord(plaintext[i]) ^ ((int(keystream[i])))
      temp.append(aux)
      #temp.append(format(aux,"02X"))   
  for j in range(i+1,len(keystream)):
      temp.append(keystream[j])
  return temp   


RC4 Decryption

In [ ]:
def RC4Decryption(ciphertext,keystream):
  temp=[]
  for i in range(len(ciphertext)):
      temp.append(chr(keystream[i]^ciphertext[i]))
  return "".join(temp) 


Check Difference

In [ ]:
def checkDifference(text1,text2):
  aux = []
  count=0
  for i in range(len(text1)):
    temp = text1[i]^text2[i]
    temp = bin(temp)[2:].rjust(8,"0")
    aux.append(temp)
    #count+=(bin(aux).count("1")) 
  string = "".join(aux)
  return string,count  
  


Analyse


In [ ]:
def analyze(text1,count):
  counter=[0]*count
  cnt = 0
  num_samples = len(text1)
  for i in range(0,len(text1)-int(math.log(count,2))):
    temp = text1[i:i+int(math.log(count,2))]
    int_val = int(temp,2)
    counter[int_val]+=1
    
  # print(counter)
  for i in range(count):
    if counter[i] != 0 :
      cnt += counter[i]
  return counter,num_samples

Flip bits


In [ ]:
def flip(text,j):
  randomlist = random.sample(range(0, len(text)), j)
  for temp in randomlist:
    text[temp]^=1
  return text  

Randomness 

In [ ]:
def randomness(counter,num_samples,num_of_counter):
  #print(num_of_counter)
  std_dev = statistics.stdev(counter)
  R = (std_dev * num_of_counter)/num_samples
  return R


Change key

In [ ]:
def change(temp2,j):
    temp2=flip(temp2,j)
    temp2=[str(aux) for aux in temp2]
    temp3=[]
    for i in range(0,len(temp2),8):
      aux=temp2[i:i+8]
      aux1="".join(aux)
      temp3.append(int(aux1,2))
    return temp3

Convert numeric list to binary string

In [ ]:
def bin_val(key1):
    temp=[]
    ans2=[]
    for aux in key1:
      temp.append(bin(aux)[2:].rjust(8,"0"))
    temp2="".join(temp)
    temp2=[int(aux) for aux in temp2]
    return temp2


Main Function

In [ ]:

if __name__ == '__main__':
    # ciphertext should be 45A01F645FC35B383552544B9BF5
    key1 = 'abcdefgh'
    # f=open("message2.txt","r")
    # plaintext=f.read()
    final=[]
    key1 = convert_key(key1)
    
    #print(keystream1)
    
    arr=[32,64,128,256,512]
    for a in arr:
      keystream1 = RC4(key1,a)
      temp2 = bin_val(key1)
      ans2 = []
      for j in range(1,33):
        ans=[]
        for k in range(100):
          temp3 =  change(temp2,j)
          keystream2 = RC4(temp3,a)
          changed,count = checkDifference(keystream1,keystream2)
          test=a
          counter,num_samples = analyze(changed,test)
          R=randomness(counter,num_samples,test)
          ans.append(R)
          temp2=[int(aux) for aux in temp2]
        ans2.append(ans)
      var=[] 
      for aux in ans2:
        var.append((sum(aux))/100)
      final.append(var)
    final=numpy.array(final)
    final=numpy.transpose(final)
    df=pd.DataFrame(final,index=[str(i) for i in range(1,33)],columns=["32","64","128","256","512"])
    print(df)
    df.to_excel('excel_16.xlsx', sheet_name='welcome')
 

     
    """     
    
    #decipheredtext=RC4Decryption(ciphertext,keystream)
    #ciphertext2=RC4Encryption(plaintext,keystream2)
    print(ciphertext)
    #print(checkDifference(ciphertext,ciphertext2))
    #print(RC4Decryption(ciphertext2,keystream2))"""
       


          32        64       128       256       512
1   1.071958  0.751795  0.475039  0.352350  0.386734
2   1.051771  0.743976  0.479767  0.355285  0.385891
3   1.060692  0.742411  0.477472  0.350471  0.386061
4   1.083170  0.765995  0.487774  0.355607  0.382240
5   1.052295  0.752609  0.472760  0.352639  0.383343
6   1.068521  0.742213  0.479605  0.352204  0.387971
7   1.069521  0.748454  0.487460  0.351875  0.383241
8   1.080890  0.748065  0.467704  0.347981  0.385093
9   1.066124  0.743934  0.485869  0.354715  0.389313
10  1.045600  0.760092  0.480468  0.351783  0.378121
11  1.073985  0.751129  0.486196  0.351277  0.377059
12  1.040724  0.751374  0.487803  0.348741  0.378192
13  1.086354  0.746749  0.484302  0.348190  0.385500
14  1.080787  0.758153  0.476630  0.353265  0.383299
15  1.067477  0.733295  0.478781  0.355281  0.383239
16  1.083649  0.736262  0.482550  0.349723  0.382357
17  1.077618  0.745811  0.478366  0.348728  0.379875
18  1.054696  0.757278  0.475230  0.353571  0.